In [38]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline

In [14]:
ids = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/id.Dennis+Schwartz.txt", sep="[\r\n]+", header=None, names=["id"])
label_3class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.3class.Dennis+Schwartz.txt", sep="[\r\n]+", names=["3class_label"])
label_4class = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/label.4class.Dennis+Schwartz.txt", sep="[\r\n]+",names=["4class_label"])
rating = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/rating.Dennis+Schwartz.txt", sep="[\r\n]+",names=["rating"])
subj = pd.read_csv("scale_data/scaledata/Dennis+Schwartz/subj.Dennis+Schwartz.txt", sep="[\r\n]+",names=["subj_extraction"])

/home/yelp_1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
/home/yelp_1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/home/yelp_1/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  

In [15]:
dennis = pd.concat([ids,label_3class, label_4class, rating, subj], axis = 1)

In [16]:
dennis.head(10)

,id,3class_label,4class_label,rating,subj_extraction
0,29420,0,0,0.1,"in my opinion , a movie reviewer's most import..."
1,17219,0,0,0.2,"you can watch this movie , that is based on a ..."
2,18406,0,0,0.2,"this is asking a lot to believe , and though i..."
3,18648,0,0,0.2,no heroes and no story are the main attributes...
4,20021,0,0,0.2,"this is not an art movie , yet i saw it an art..."
5,20454,0,0,0.2,a satirical film where warren beatty tries to ...
6,20473,0,0,0.2,this sci-fi'er left a lot to be desired . it d...
7,20538,0,0,0.2,"this is a dull , unfunny film , lacking an edg..."
8,21002,0,0,0.2,instead he makes him an offer he can't very we...
9,21739,0,0,0.2,vibrant culture . great people . thinking of h...


In [17]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(dennis, test_size=0.2)

In [18]:
num_words = 2000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train['subj_extraction'].values)

In [24]:
X_train = tokenizer.texts_to_sequences(train['subj_extraction'].values)
X_train = pad_sequences(X_train, maxlen=2000)

In [25]:
y_train = train['4class_label']
y_test = test['4class_label']

In [33]:
y_train = to_categorical(y_train, num_classes=4)
y_test = to_categorical(y_test, num_classes=4)

In [50]:
X_train

array([[   0,    0,    0, ...,   19, 1510,  904],
       [   0,    0,    0, ...,    6,    1,   27],
       [   0,    0,    0, ...,  657,  123,   21],
       ...,
       [   0,    0,    0, ...,  163,  804,   11],
       [   0,    0,    0, ...,    8,   14,  386],
       [   0,    0,    0, ...,  136,  560, 1447]], dtype=int32)

In [51]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(num_words, embedding_vecor_length, input_length=X_train.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=10, batch_size=64)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 2000, 32)          64000     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 404       
Total params: 117,604
Trainable params: 117,604
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
821/821 [==============================] - 39s 47ms/step - loss: 1.3504 - categorical_accuracy: 0.4080
Epoch 2/10
821/821 [==============================] - 37s 45ms/step - loss: 1.2608 - categorical_accuracy: 0.4397
Epoch 3/10
821/821 [==============================] - 37s 45ms/step - loss: 1.2634 - categorical_accuracy: 0.4397
Epoch 4/10
821/821 [==============================] -

In [52]:
X_test = tokenizer.texts_to_sequences(test['subj_extraction'].values)
X_test = pad_sequences(X_test, maxlen=2000)

In [53]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 36.89%


In [51]:
train["pos"]=dennis.apply(lambda x:int(x["rating"]>0.5), axis=1)
test["pos"]=dennis.apply(lambda x:int(x["rating"]>0.5), axis=1)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
